In [ ]:
import pandas as pd
import json
from pandas import Series,DataFrame
import re
import datetime
from bs4 import BeautifulSoup
import requests
#寻找小区对应的区域和建造年份
def get_data(x):
    try:
        html=requests.get('https://dl.lianjia.com/xiaoqu/rs'+x)
        soup=BeautifulSoup(html.text,'lxml')
        s=soup.select('.info .title a')
        for i in range(0,len(s)):
            if s[i].string==x:
                district=soup.select('.info .positionInfo .district')[i].string
                bizcircle=soup.select('.info .positionInfo .bizcircle')[i].string
                temp=soup.select('.info .positionInfo')[i].get_text()
                nianfen=re.search('\d+',temp).group()
                return x+' '+district+' '+bizcircle+' '+ str(nianfen)
        else:
            return None
    except IndexError as e:
        print(x)

pd.set_option('display.max_rows',None)
# #设置显示全部列，不省略
pd.set_option('display.max_columns',None)

item=[]
for line in open('data/lianjia.json','r',encoding='utf-8'):
    item.append(json.loads(line))
    
df=DataFrame(item)
df.drop_duplicates(inplace=True)

 

In [ ]:
getarea=lambda x: float(re.search(r"\d+(\.\d+)?",x).group())
df['area']=df['area'].apply(getarea)
tofloat=lambda x:float(x)
df['unitPrice']=df['unitPrice'].apply(tofloat)
df['totalPrice']=df['totalPrice'].apply(tofloat)
df['dealDate']=pd.to_datetime(df['dealDate'])
df['dealCycle']=df['dealCycle'].map(lambda x:int(re.search('\d+',x).group()))
df['guaPai']=df['guaPai'].apply(getarea)

In [ ]:
#找出降价幅度大的房子
f=df[((df['guaPai']-df['totalPrice'])/df['guaPai']>0.05) & (df['dealDate'].map(lambda x:x.year)==2021)].sort_values(by=['title','dealDate'])
f['chazhi']=round((f['guaPai']-f['totalPrice'])/f['guaPai'],2)

In [ ]:
#找出各个销售的销售量，目前看工人村单个销售卖的还真多
s=df.groupby(['seller'])[['seller']].count()
s.index.name='num'
s.sort_values(by=['seller'],ascending=False)
s.head()

In [ ]:
#找出各小区的销售数量，先忽略时间
# xiaoqu=df.groupby(['title'])[['title']].count()
xiaoqu=df['title'].groupby([df['title'],df['dealDate'].apply(lambda x:x.year)]).count()
xiaoqu.index.names=['title','year']
result=list(set(list(map(lambda x:re.search('悦泰(\w+)?',x),xiaoqu.index.levels[0]))))
result.remove(None)
r=[i.group() for i in result]


In [ ]:
s1=xiaoqu.loc[r[1]]
s2=xiaoqu.loc[r[0]]
s1.name=r[1]
s2.name=r[0]
print(s1,s2)


In [ ]:
#找出某个小区的销售情况
print(get_data('第五郡五号地'))
df_wu=df[df['title'].apply(lambda x: '第五郡' in x and '公寓' not in x)]
df_wu=df_wu.sort_values(by='dealDate')
df_wu=df_wu.reset_index(drop=True)
func=lambda x:str((x.year)*100+(x.month))[0:4]+'-'+str((x.year)*100+(x.month))[4:]
df_wu['month']=df_wu['dealDate'].map(func)
df_wu['title'].groupby(df_wu['month']).count()
# df[df['title']=='金海花园西园'][['title']].count()

In [ ]:
#找出小区每年均价，按月标本太少，不太准确
df_wu['unitPrice'].groupby(df_wu['dealDate'].map(lambda x:x.year)).describe()

In [ ]:
#找出小区的成交周期
df_wu['dealCycle'].groupby(df_wu['dealDate'].apply(lambda x:x.year)).describe()

In [ ]:
#找出房子里只有一天成交周期的房子
df[(df['dealCycle'].map(lambda x:x==1)) & (df['dealDate'].apply(lambda x:x.year)==2021)].head()

In [ ]:
#备用功能        
# list=[]
# for i in df['title'].drop_duplicates():
#     g=get_data(i)
#     if g is not None:
#         list.append(g)

# %run main.py
#df.to_json(path_or_buf='data/test.json',orient='index',force_ascii=False)
#检查area数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search(r'\d+(\.\d+)?(平米)',result[2]) is None:
#         print(df.iloc[i])
#         df.drop([i])
# #检查挂牌数据
# for i in range(0,len(df)):
#     result=df.iloc[i]
#     if re.search('挂牌*',result[7]) is None:
#         print(str(i))
        # df.drop([i])

In [16]:
dd=df['unitPrice'].groupby(df['dealDate'].apply(lambda x:x.year)).describe()
dd

,count,mean,std,min,25%,50%,75%,max
dealDate,,,,,,,,
2007,1.0,6526.000000,NaN,6526.0,6526.00,6526.0,6526.00,6526.0
2009,2.0,6819.500000,2312.946281,5184.0,6001.75,6819.5,7637.25,8455.0
2010,5.0,12219.400000,2995.088863,9944.0,10399.00,10657.0,12934.00,17163.0
2011,311.0,13109.012862,3712.703479,7827.0,10591.00,12119.0,14520.50,34729.0
2012,969.0,11756.955624,3071.132418,5073.0,9640.00,10968.0,13086.00,27738.0
2013,1196.0,12435.205686,3236.581713,6352.0,10097.00,11500.0,14180.50,27496.0
2014,809.0,11971.007417,3237.322643,5604.0,9626.00,11032.0,13951.00,27103.0
2015,1823.0,11374.232035,3576.385799,4266.0,8858.50,10365.0,13549.00,34846.0
2016,7730.0,10114.846572,4165.222990,3050.0,7866.50,9264.5,11272.75,128409.0


In [19]:
dd[dd.index>2015]

,count,mean,std,min,25%,50%,75%,max
dealDate,,,,,,,,
2016,7730.0,10114.846572,4165.222990,3050.0,7866.5,9264.5,11272.75,128409.0
2017,12410.0,10842.420226,4336.281539,1834.0,8166.0,9933.5,12299.75,105216.0
2018,12589.0,12033.056160,4818.683399,3487.0,8740.0,11006.0,13962.00,94424.0
2019,15852.0,12854.646417,5076.039573,2008.0,9217.0,12076.5,14839.00,60580.0
2020,17915.0,13691.954228,5876.524765,1705.0,9608.0,12903.0,15867.50,65225.0
2021,10511.0,14895.846542,6064.318374,2.0,10454.5,14103.0,17367.50,61788.0
